In [220]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

In [221]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.int)

print(bits_info)

tensor([[0, 0, 1, 1],
        [0, 0, 1, 0],
        [1, 1, 0, 0],
        [1, 0, 1, 0],
        [1, 0, 0, 1],
        [0, 1, 1, 0],
        [0, 0, 1, 0],
        [1, 1, 1, 0],
        [0, 1, 1, 0],
        [1, 1, 1, 1],
        [0, 0, 0, 1],
        [0, 0, 1, 1],
        [1, 0, 0, 1],
        [1, 1, 1, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [1, 1, 0, 0],
        [0, 0, 1, 1],
        [1, 0, 0, 1],
        [0, 1, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 1, 0],
        [0, 0, 1, 0],
        [0, 0, 1, 0],
        [1, 0, 1, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [1, 0, 0, 0],
        [0, 1, 0, 1],
        [1, 1, 0, 0],
        [1, 0, 1, 1],
        [1, 0, 1, 0],
        [0, 0, 1, 1],
        [0, 1, 1, 0],
        [0, 0, 1, 1],
        [1, 1, 0, 0],
        [0, 1, 0, 1],
        [1, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 1],
        [0, 1, 1, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 0, 1, 0],
        [1, 1, 1, 0],
        [0

Hamming(7,4) Encoder

In [222]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.int)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [223]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[0, 0, 1, 1, 1, 0, 0],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [1, 0, 0, 1, 0, 0, 1],
        [0, 1, 1, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 1, 1, 0, 0, 0, 0],
        [0, 1, 1, 0, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [1, 1, 1, 0, 0, 0, 0],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 1, 0, 0, 0, 1, 1],
        [0, 0, 1, 1, 1, 0, 0],
        [1, 0, 0, 1, 0, 0, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [1, 0, 1, 0, 1, 0, 1],
        [0, 1, 1, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 1, 1],
        [0, 0, 1, 0, 0, 1, 1],
        [1, 0, 1, 1, 0, 1, 0],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 0, 1, 1, 1, 1],
        [1, 0, 0, 0, 1, 1, 0],
        [0, 1, 0, 1, 0, 1, 0],
        [1, 1, 0, 0, 0, 1, 1],
        [1, 0, 1, 1, 0, 1, 0],
        [1, 0, 1, 0, 1, 0, 1],
        

BPSK Modulator + Noise

In [224]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

    Args:
        codeword: data received from the Hamming(7,4) encoder(Tensor)

    Returns:
        data: Tensor contain all data modulated and add noise
    """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, codeword, snr_dB):
        
        # data = torch.tensor(data, dtype=float)
        data = codeword.to(dtype=torch.float)
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(snr_dB / 10))
            noise = torch.sqrt(1/(2*noise_power)) * torch.randn(len(bits))
            noised_signal = bits + noise
            # noised_signal = bits
            data[i] = noised_signal
    
       
        return data

In [225]:
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
modulated_noise_signal = modulator(encoded_codeword, snr_dB)
print(modulated_noise_signal)

tensor([[-0.9954, -0.9189,  0.9975,  0.6954,  0.9574, -0.8343, -0.9457],
        [-1.1238, -1.0599,  0.9207, -0.7988, -0.9620,  0.8060,  1.1414],
        [ 0.9936,  1.1384, -1.0278, -1.1659, -1.0672,  1.1682,  1.0260],
        [ 0.8998, -0.8425,  0.7621, -1.0514,  0.8986, -0.9155,  1.0738],
        [ 1.0931, -0.8998, -1.1288,  1.2566, -0.7871, -0.9671,  1.0074],
        [-1.0936,  0.8880,  1.1507, -0.8176,  1.0226,  1.0856, -0.9375],
        [-0.9760, -1.1077,  1.1180, -0.8508, -0.7016,  1.2263,  1.0464],
        [ 0.8784,  0.9145,  1.0082, -1.1153, -0.7020, -1.1000, -0.8331],
        [-0.9034,  1.2408,  0.9278, -1.1211,  1.0246,  0.9356, -0.7464],
        [ 1.0471,  0.8679,  0.9717,  1.0836,  1.0257,  1.0455,  0.9320],
        [-0.8002, -1.0827, -0.8594,  1.0607,  0.8988,  1.2013,  1.2509],
        [-1.0417, -1.0431,  0.7857,  1.0686,  0.9382, -0.8365, -1.0609],
        [ 1.0697, -0.9415, -1.1239,  0.8113, -1.0992, -1.0377,  1.0821],
        [ 1.1127,  0.7890,  0.9515, -1.1206, -1.262

LLR Log-likelihood
y = s + n
Assuming that \( s \) is equally likely to be 0 or 1, and \( n \) is Gaussian with zero mean and variance \( N_0/2 \), where \( N_0 \) is the noise power spectral density.


In [226]:
def llr(signal, snr):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal from BPSK.
        noise_std (float): Standard deviation of the noise.

    Returns:
        llr: Log Likelihood Ratio (LLR) values.
    """
    
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    noise_std = torch.sqrt(torch.tensor(10**(snr / 10)))

    # Calculate the LLR
    llr = 2 * signal * noise_std

    # return llr_values, llr
    return llr


In [227]:
llr_output = llr(modulated_noise_signal, snr_dB)
print("LLR values:", llr_output)

LLR values: tensor([[-11.1950, -10.3353,  11.2191,   7.8212,  10.7674,  -9.3828, -10.6364],
        [-12.6393, -11.9206,  10.3544,  -8.9845, -10.8192,   9.0650,  12.8369],
        [ 11.1749,  12.8039, -11.5597, -13.1125, -12.0029,  13.1391,  11.5391],
        [ 10.1199,  -9.4752,   8.5709, -11.8254,  10.1060, -10.2967,  12.0764],
        [ 12.2935, -10.1201, -12.6959,  14.1324,  -8.8522, -10.8764,  11.3296],
        [-12.2995,   9.9874,  12.9415,  -9.1954,  11.5011,  12.2093, -10.5441],
        [-10.9764, -12.4577,  12.5734,  -9.5686,  -7.8904,  13.7924,  11.7688],
        [  9.8791,  10.2849,  11.3386, -12.5438,  -7.8947, -12.3720,  -9.3696],
        [-10.1599,  13.9550,  10.4343, -12.6091,  11.5235,  10.5225,  -8.3945],
        [ 11.7761,   9.7608,  10.9288,  12.1875,  11.5354,  11.7585,  10.4820],
        [ -8.9998, -12.1773,  -9.6653,  11.9295,  10.1087,  13.5109,  14.0685],
        [-11.7156, -11.7315,   8.8366,  12.0186,  10.5520,  -9.4081, -11.9318],
        [ 12.0308, -10.5889,

LDPC Decoder

In [228]:
class LDPCBeliefPropagation(torch.nn.Module):
    def __init__(self, H):
        """
        LDPC Belief Propagation.
    
        Args:
            H: Low density parity code for building tanner graph.
            llr: Log Likelihood Ratio (LLR) values. Only for 7-bit codeword.
    
        Returns:
            estimated_bits: the output result from belief propagation.
        """
        
        super(LDPCBeliefPropagation, self).__init__()
        self.H = H
        self.num_check_nodes, self.num_variable_nodes = H.shape

        # Initialize messages
        self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes),dtype=torch.float)
        self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes),dtype=torch.float)

    def forward(self, llr, max_iter):
        for iteration in range(max_iter):
            # Variable to check node messages
            for i in range(self.num_variable_nodes):
                for j in range(self.num_check_nodes):
                    # Compute messages from variable to check nodes
                    connected_checks = self.H[j, :] == 1
                    product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, j]))
                    self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product

            # Check to variable node messages
            for i in range(self.num_check_nodes):
                for j in range(self.num_variable_nodes):
                    # Compute messages from check to variable nodes
                    connected_vars = self.H[:, j] == 1
                    sum_msgs = torch.sum(self.messages_c_to_v[connected_vars, i]) - self.messages_v_to_c[j, i]
                    self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))

        # Calculate the final estimated bits and only take first four bits
        estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
        estimated_bits = torch.where(estimated_bits>0, torch.tensor(1), torch.tensor(0))
        estimated_bits = estimated_bits[0:4]


        return estimated_bits
    

In [230]:
# Define LDPC parameters
H = torch.tensor([ [1, 1, 1, 0, 0, 0, 0],
                   [0, 0, 1, 1, 1, 0, 0],
                   [0, 1, 0, 0, 1, 1, 0],
                   [1, 0, 0, 1, 0, 0, 1],])
iter = 10
ldpc_bp = LDPCBeliefPropagation(H)

# Store the final result from LDPC
tensor_size = torch.Size([llr_output.shape[0], 4])
final_result = torch.zeros(tensor_size)

#loop all the llr and get result.
for i in range(llr_output.shape[0]):
    bp_data = llr_output[i]
    estimated_bits = ldpc_bp(bp_data, iter)
    final_result[i] = estimated_bits

print(final_result)

tensor([[0., 0., 1., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [1., 0., 1., 0.],
        [1., 0., 0., 1.],
        [0., 1., 1., 0.],
        [0., 0., 1., 0.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [1., 0., 0., 1.],
        [1., 1., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 1., 1.],
        [1., 0., 0., 1.],
        [0., 1., 0., 0.],
        [1., 0., 1., 0.],
        [0., 1., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [1., 0., 1., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 0.],
        [0., 1., 0., 1.],
        [1., 1., 0., 0.],
        [1., 0., 1., 1.],
        [1., 0., 1., 0.],
        [0., 0., 1., 1.],
        [0., 1., 1., 0.],
        [0., 0., 1., 1.],
        [1., 1., 0., 0.],
        [0., 1., 0., 1.],
        [1., 0., 0., 0.],
        [1.,

Strange behavior recording:
In 100 7-bit codewords, the speed on MPS(GPU) is slower than CPU. Reason unknown.

In [ ]:
# class LDPCBeliefPropagation(torch.nn.Module):
#     def __init__(self, H):
#         """
#         LDPC Belief Propagation.
#
#         Args:
#             H: Low density parity code for building tanner graph.
#             llr: Log Likelihood Ratio (LLR) values. Only for 7-bit codeword.
#
#         Returns:
#             estimated_bits: the output result from belief propagation.
#         """
#
#         super(LDPCBeliefPropagation, self).__init__()
#         self.H = H
#         self.num_check_nodes, self.num_variable_nodes = H.shape
#
#         # Initialize messages
#         self.messages_v_to_c = torch.ones((self.num_variable_nodes, self.num_check_nodes), dtype=torch.float).to(
#             mps_device)
#         self.messages_c_to_v = torch.zeros((self.num_check_nodes, self.num_variable_nodes), dtype=torch.float).to(
#             mps_device)
#
#     def forward(self, llr, max_iter):
#         for iteration in range(max_iter):
#             # Variable to check node messages
#             for i in range(self.num_variable_nodes):
#                 for j in range(self.num_check_nodes):
#                     # Compute messages from variable to check nodes
#                     connected_checks = self.H[j, :] == 1
#                     product = torch.prod(torch.tanh(0.5 * self.messages_v_to_c[connected_checks, j]))
#                     self.messages_v_to_c[i, j] = torch.sign(llr[i]) * product
#
#             # Check to variable node messages
#             for i in range(self.num_check_nodes):
#                 for j in range(self.num_variable_nodes):
#                     # Compute messages from check to variable nodes
#                     connected_vars = self.H[:, j] == 1
#                     sum_msgs = torch.sum(self.messages_c_to_v[connected_vars, i]) - self.messages_v_to_c[j, i]
#                     self.messages_c_to_v[i, j] = 2 * torch.atan(torch.exp(0.5 * sum_msgs))
#
#         # Calculate the final estimated bits and only take first four bits
#         estimated_bits = torch.sign(llr) * torch.prod(torch.tanh(0.5 * self.messages_c_to_v), dim=0)
#         tensor_1 = torch.tensor(1, device=mps_device)
#         tensor_0 = torch.tensor(0, device=mps_device)
#         estimated_bits = torch.where(estimated_bits > 0, tensor_1, tensor_0)
#         estimated_bits = estimated_bits[0:4]
#
#         return estimated_bits

In [ ]:
# # Define LDPC parameters
# H = torch.tensor([[1, 1, 1, 0, 0, 0, 0],
#                   [0, 0, 1, 1, 1, 0, 0],
#                   [0, 1, 0, 0, 1, 1, 0],
#                   [1, 0, 0, 1, 0, 0, 1], ], device=mps_device)
# iter = 1
# ldpc_bp = LDPCBeliefPropagation(H)
#
# # Store the final result from LDPC
# tensor_size = torch.Size([llr_output.shape[0], 4])
# final_result = torch.zeros(tensor_size).to(mps_device)
#
# llr_output = llr_output.to(mps_device)
#
# #loop all the llr and get result.
# for i in range(llr_output.shape[0]):
#     bp_data = llr_output[i]
#     estimated_bits = ldpc_bp(bp_data, iter)
#     final_result[i] = estimated_bits
#
# print(final_result)
#
# # llr_demodulator_output = llr_output
# #
# # estimated_bits = ldpc_bp(llr_demodulator_output, iter)
# #
# # print("LLR Demodulator Output:", llr_demodulator_output)
# # print("Estimated Bits:", estimated_bits)

Comparation and Plot